In [1]:
using DataFrames, CSV, Plots, Random, StatsBase

# DecisionTree

In [2]:
using DecisionTree

In [4]:
data = CSV.read("Data//melb_data.csv", DataFrame);
println( "number of read data points $(size(data)[1])")
#fdata = dropmissing(data);
dropmissing!(data);
filtered = convert( Array, data[!,[:Rooms, :Bathroom, :Landsize, :BuildingArea, :YearBuilt, :Lattitude, :Longtitude]] );
y = convert( Array, data[!,:Price] );
println( "number of valid data points $(size(filtered)[1])")

number of read data points 13580
number of valid data points 6196


In [5]:
println( typeof(filtered) )
println( typeof(y) )
#first(filtered,5)
size(filtered)

Array{Float64,2}
Array{Float64,1}


(6196, 7)

In [6]:
@time idx = shuffle!(collect(1:size(filtered)[1]));
sp = convert(Int, length(idx)*0.75)
trainX = filtered[idx[1:sp],:];
testX  = filtered[idx[sp+1:end],:];
trainY = y[idx[1:sp]];
testY  = y[idx[sp+1:end]];
println( "train set size ", size(trainX)[1])
println( "test  set size ", size(testX)[1])

  0.162061 seconds (887.87 k allocations: 43.629 MiB, 6.03% gc time)
train set size 4647
test  set size 1549


In [18]:
mean_squared_error(x)  = mean( x.^2 );
mean_absolute_error(x) = mean( abs.(x) );

In [27]:
model = DecisionTreeRegressor();#max_depth=7);

In [28]:
@time DecisionTree.fit!(model, trainX, trainY)

  0.019542 seconds (4.55 k allocations: 657.484 KiB)


DecisionTreeRegressor
max_depth:                -1
min_samples_leaf:         5
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
root:                     Decision Tree
Leaves: 754
Depth:  22

In [25]:
@time pred = DecisionTree.predict(model, testX);
println( "prediction MSE ", mean_squared_error( pred - testY ) )
println( "prediction MAE ",mean_absolute_error( pred - testY ) )

  0.000299 seconds (1.55 k allocations: 242.328 KiB)
prediction MSE 1.8990260343369193e11
prediction MAE 246078.65335232046


In [26]:
@time predT = DecisionTree.predict(model, trainX);
println( "train set prediction MSE ", mean_squared_error( predT - trainY ) )
println( "train set prediction MAE ",mean_absolute_error( predT - trainY ) )

  0.001040 seconds (4.65 k allocations: 726.266 KiB)
train set prediction MSE 1.0293854404901884e11
train set prediction MAE 199061.7166734019


# Random forest

In [29]:
rf = RandomForestClassifier()

RandomForestClassifier
n_trees:             10
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             nothing
ensemble:            nothing

In [31]:
DecisionTree.fit!(rf, filtered, y)

RandomForestClassifier
n_trees:             10
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [131000.0, 145000.0, 170000.0, 185000.0, 200000.0, 210000.0, 216000.0, 220000.0, 222000.0, 227000.0  …  5.5e6, 5.51e6, 5.525e6, 5.6e6, 5.7e6, 6.25e6, 6.5e6, 7.65e6, 8.0e6, 9.0e6]
ensemble:            Ensemble of Decision Trees
Trees:      10
Avg Leaves: 3111.5
Avg Depth:  26.5

In [32]:
@time pred = DecisionTree.predict(rf, testX);
println( "prediction MSE ", mean_squared_error( pred - testY ) )
println( "prediction MAE ",mean_absolute_error( pred - testY ) )

  0.077973 seconds (210.07 k allocations: 11.385 MiB)
prediction MSE 5.216511048846574e10
prediction MAE 104216.9338928341


# MLJ

In [12]:
using MLJ

# ScikitLearn

In [13]:
using ScikitLearn
using ScikitLearn.CrossValidation: train_test_split

In [14]:
trainX1, testX1, trainY1, testY1 = train_test_split(filtered, y, test_size=0.25, random_state=42);